In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
df_train = pd.read_csv("/kaggle/input/hr-analytics-job-change-of-data-scientists/aug_train.csv", header = 0)
df_test = pd.read_csv("/kaggle/input/hr-analytics-job-change-of-data-scientists/aug_test.csv", header = 0)

In [ ]:
df_train.info()

In [ ]:
df_test.info()

In [ ]:
## in order to change city column to numerical value:
city = df_train['city']
lst  = []
for temp in city:
    s = temp.split("_")
    temp = s[1]
    lst.append(int(temp))
for i in range(0, len(city)):
    city[i] = lst[i]

In [ ]:
## in order to change city column to numerical value:
city_t = df_test['city']
lst  = []
for temp in city_t:
    s = temp.split("_")
    temp = s[1]
    lst.append(int(temp))
for i in range(0, len(city_t)):
    city_t[i] = lst[i]

In [ ]:
## for experience
df_train.experience[df_train.experience == ">20"] = "21"
df_train.experience[df_train.experience == "<1"] = "0"

df_test.experience[df_test.experience == ">20"] = "21"
df_test.experience[df_test.experience == "<1"] = "0"

## for last_new_job
df_train.last_new_job[df_train.last_new_job == "never"] = "0"
df_train.last_new_job[df_train.last_new_job == ">4"] = "5"

df_test.last_new_job[df_test.last_new_job == "never"] = "0"
df_test.last_new_job[df_test.last_new_job == ">4"] = "5"

In [ ]:
df_train.city = city.astype(int)
df_test.city  = city_t.astype(int)

In [ ]:
column = ["gender", "enrolled_university", "education_level",
          "major_discipline", "experience", "company_size", "company_type", "last_new_job"]
for column in column:
    df_train[column] = df_train[column].fillna(df_train[column].mode()[0])

In [ ]:
## converting experience, and last_new_jobs to int
df_train["experience"] = df_train["experience"].astype(int)
df_train["last_new_job"] = df_train["last_new_job"].astype(int)

In [ ]:
df_train.info()

In [ ]:
# dummy variables
df_train = pd.get_dummies(df_train, columns = ["gender", "relevent_experience", "enrolled_university", "education_level",
                                               "major_discipline", "company_size", "company_type"],
                          drop_first = True)

In [ ]:
df_train.info()

In [ ]:
del df_train['enrollee_id']

In [ ]:
## training the model
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV

In [ ]:
X_train = df_train.loc[:, df_train.columns != "target"] 
y = df_train["target"]

In [ ]:
scalar = StandardScaler().fit(X_train)
X_train = scalar.transform(X_train)

In [ ]:
##logistic regtression
clf_log = LogisticRegression()

clf_log.fit(X_train, y)

from sklearn.metrics import accuracy_score, confusion_matrix

confusion_matrix(y, clf_log.predict(X_train))

In [ ]:
accuracy_score(y, clf_log.predict(X_train))

In [ ]:
df_test["gender"] = df_test["gender"].fillna(df_test["gender"].mode()[0])
df_test["enrolled_university"] = df_test["enrolled_university"].fillna(df_test["enrolled_university"].mode()[0])
df_test["education_level"] = df_test["education_level"].fillna(df_test["education_level"].mode()[0])
df_test["major_discipline"] = df_test["major_discipline"].fillna(df_test["major_discipline"].mode()[0])
df_test["experience"] = df_test["experience"].fillna(df_test["experience"].mode()[0])
df_test["company_size"] = df_test["company_size"].fillna(df_test["company_size"].mode()[0])
df_test["company_type"] = df_test["company_type"].fillna(df_test["company_type"].mode()[0])
df_test["last_new_job"] = df_test["last_new_job"].fillna(df_test["last_new_job"].mode()[0])

In [ ]:
df_test.info()

In [ ]:
## converting experience, and last_new_jobs to int
df_test["experience"] = df_test["experience"].astype(int)
df_test["last_new_job"] = df_test["last_new_job"].astype(int)

In [ ]:
# dummy variables
df_test = pd.get_dummies(df_test, columns = ["gender", "relevent_experience", "enrolled_university", "education_level",
                                               "major_discipline", "company_size", "company_type"],
                          drop_first = True)

In [ ]:
df_test.info()

In [ ]:
X_test = df_test.loc[:, df_test.columns != "enrollee_id"]
scalar_test = StandardScaler().fit(X_test)
X_test = scalar_test.transform(X_test)

In [ ]:
X_test.shape

In [ ]:
X_train.shape

In [ ]:
y_pred = clf_log.predict(X_test)

In [ ]:
data = pd.DataFrame(df_test["enrollee_id"])

In [ ]:
data["target"] = y_pred

In [ ]:
data.to_csv("HR Analytics.csv")

In [ ]:
sns.countplot(x = "target", data = data)